In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"
env = "staging"

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)
date_day = "2020-05-*"
hr = "*" #now().hour-1

rrm_radar_path = s3_bucket + "rrm_radar_channel_stat/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)
print(rrm_radar_path)


date_day = "2020-05-*"
curr_hr = '*'  # latest 
ap_neighbors_path = s3_bucket + "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

print(ap_neighbors_path)


s3://mist-aggregated-stats-staging/aggregated-stats/rrm_radar_channel_stat/dt=2020-05-*/hr=*/*.csv
s3://mist-aggregated-stats-staging/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-05-*/hr=*/*.csv


In [2]:
# rdd_rrm_local = spark.sparkContext.sequenceFile(rrm_local_path)
rdd_rrm_events = spark.sparkContext.textFile(rrm_radar_path)
rdd_rrm_events.take(10)


['site_id,ap_id,pre_channel,radar_channel_count,last_radar_time,time',
 '70e0f468-fc13-11e5-85ad-0242ac110008,5c5b350eb3f2,122,1,1588293468,1588291200',
 'site_id,ap_id,pre_channel,radar_channel_count,last_radar_time,time',
 'a8178443-ecb5-461c-b854-f16627619ab3,5c5b353e4f51,52,1,1588309135,1588305600',
 'a8178443-ecb5-461c-b854-f16627619ab3,5c5b353e5393,108,1,1588306596,1588305600',
 'site_id,ap_id,pre_channel,radar_channel_count,last_radar_time,time',
 'd7c802c4-1d3a-4aa5-90fa-afd14ac7e089,5c5b352e20a8,0,1,1588327560,1588327200',
 'site_id,ap_id,pre_channel,radar_channel_count,last_radar_time,time',
 'a8178443-ecb5-461c-b854-f16627619ab3,5c5b353e4e84,116,1,1588337189,1588334400',
 'site_id,ap_id,pre_channel,radar_channel_count,last_radar_time,time']

In [3]:
df_radar = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(rrm_radar_path)
df_radar.createOrReplaceTempView("scan_data")
df_Schema = df_radar.schema

In [4]:
df_radar.show(5)

+--------------------+------------+-----------+-------------------+---------------+----------+
|             site_id|       ap_id|pre_channel|radar_channel_count|last_radar_time|      time|
+--------------------+------------+-----------+-------------------+---------------+----------+
|c943d077-1b67-491...|5c5b35500622|        104|                  1|     1589096727|1589094000|
|c943d077-1b67-491...|5c5b35500924|        116|                  1|     1589095299|1589094000|
|c943d077-1b67-491...|5c5b35500686|        104|                  1|     1589095247|1589094000|
|c943d077-1b67-491...|5c5b35500587|         64|                  1|     1589096287|1589094000|
|c943d077-1b67-491...|5c5b35500839|         64|                  1|     1588919882|1588917600|
+--------------------+------------+-----------+-------------------+---------------+----------+
only showing top 5 rows



# Impacted Sites

In [5]:
#impacted Sites
df_radar_sites = df_radar.select('site_id', 'radar_channel_count').groupBy("site_id").count()
df_radar_sites.show()

+--------------------+-----+
|             site_id|count|
+--------------------+-----+
|ee915a44-b963-11e...|   11|
|f688779c-e335-4f8...|    3|
|70e0f468-fc13-11e...|    1|
|e80749be-963c-4d2...|    2|
|6db15e18-bc94-11e...|    2|
|67970e46-4e12-11e...|   12|
|fc656275-b157-43f...|    1|
|9c3f2be6-397d-11e...|    2|
|35670395-458f-4ec...|    2|
|d23f418c-a5ab-4e3...|    6|
|a8178443-ecb5-461...|   21|
|20325077-2d75-4fa...|    2|
|96ca2c79-b7db-402...|    1|
|d7c802c4-1d3a-4aa...|    9|
|12c07c11-51a3-495...|    3|
|f733a5d7-5179-4df...|    2|
|fe1f336e-fba7-44d...|    1|
|d59ee965-e757-45a...|    1|
|b0bbfdc0-aaba-11e...|    1|
|4a6427ac-4514-4ab...|    1|
+--------------------+-----+
only showing top 20 rows



In [6]:
impacted_sites = df_radar_sites.filter(col('count')>10).select("site_id", "count").collect()
impacted_sites

[Row(site_id='ee915a44-b963-11e5-b14c-1258369c38a9', count=11),
 Row(site_id='67970e46-4e12-11e6-9188-0242ac110007', count=12),
 Row(site_id='a8178443-ecb5-461c-b854-f16627619ab3', count=21),
 Row(site_id='c943d077-1b67-4915-a690-e698543d2790', count=114)]

# One Impacted-site

In [7]:
impacted_site_id = impacted_sites[0]['site_id']

In [8]:
#impacted APs
df_radar_in_site = df_radar.filter(col("site_id")==impacted_site_id).select('ap_id',col("pre_channel").alias("radar_channel"), 'last_radar_time')

df_radar_in_site.show()
# df_radar_ap_lastseen.show() )

# df_radar_ap_lastseen = df_radar_in_site.select('pre_channel', 'last_radar_time').groupBy("ap_id", col("pre_channel").alias("radar_channel"), 'last_radar_time').max()
# df_radar_ap_lastseen.show()

# df_radar_ap_lastseen = df_radar.filter(col("site_id")==impacted_site_id).select("ap_id", 'pre_channel', 'last_radar_time').groupBy("ap_id", col("pre_channel").alias("radar_channel"), 'last_radar_time').max()
# df_radar_ap_lastseen.show()

+------------+-------------+---------------+
|       ap_id|radar_channel|last_radar_time|
+------------+-------------+---------------+
|5c5b3550032f|          100|     1588928836|
|5c5b3550032f|          100|     1588938523|
|5c5b3550032f|          100|     1588934289|
|5c5b3550032f|          132|     1588948689|
|5c5b3550032f|          132|     1589037054|
|5c5b3550032f|           52|     1589209332|
|5c5b3550032f|           52|     1588952779|
|5c5b3550032f|          132|     1588969154|
|5c5b3550032f|           52|     1588960942|
|5c5b3550032f|          116|     1588963397|
|5c5b3550032f|          100|     1589019556|
+------------+-------------+---------------+



In [12]:
from datetime import datetime
from pyspark.sql.types import IntegerType

timestamp = 1589019556
get_hour = udf(lambda t: datetime.fromtimestamp(t).hour, IntegerType())
get_weekday = udf(lambda t: datetime.fromtimestamp(t).weekday(), IntegerType())
df_radar_in_site= df_radar_in_site.withColumn("hour", get_hour(col("last_radar_time")))\
                                  .withColumn("dayofweek", get_weekday(col("last_radar_time")))

df_radar_in_site.select("ap_id", 'radar_channel', "dayofweek", "hour")\
.groupBy("ap_id", 'radar_channel', "dayofweek", "hour").count().show()


+------------+-------------+---------+----+-----+
|       ap_id|radar_channel|dayofweek|hour|count|
+------------+-------------+---------+----+-----+
|5c5b3550032f|           52|        0|   8|    1|
|5c5b3550032f|          132|        4|  13|    1|
|5c5b3550032f|           52|        4|   8|    1|
|5c5b3550032f|          100|        4|   3|    1|
|5c5b3550032f|          100|        5|   3|    1|
|5c5b3550032f|           52|        4|  11|    1|
|5c5b3550032f|          116|        4|  11|    1|
|5c5b3550032f|          100|        4|   2|    1|
|5c5b3550032f|          132|        5|   8|    1|
|5c5b3550032f|          100|        4|   4|    1|
|5c5b3550032f|          132|        4|   7|    1|
+------------+-------------+---------+----+-----+



In [13]:
df_radar_in_site.select("ap_id", 'radar_channel', "dayofweek", "hour")\
.groupBy("ap_id", 'radar_channel', "dayofweek", "hour").count().show()

+------------+-------------+---------+----+-----+
|       ap_id|radar_channel|dayofweek|hour|count|
+------------+-------------+---------+----+-----+
|5c5b3550032f|           52|        0|   8|    1|
|5c5b3550032f|          132|        4|  13|    1|
|5c5b3550032f|           52|        4|   8|    1|
|5c5b3550032f|          100|        4|   3|    1|
|5c5b3550032f|          100|        5|   3|    1|
|5c5b3550032f|           52|        4|  11|    1|
|5c5b3550032f|          116|        4|  11|    1|
|5c5b3550032f|          100|        4|   2|    1|
|5c5b3550032f|          132|        5|   8|    1|
|5c5b3550032f|          100|        4|   4|    1|
|5c5b3550032f|          132|        4|   7|    1|
+------------+-------------+---------+----+-----+



## Impacted APs, channel

In [14]:
df_radar_by_ap = df_radar_in_site.select('ap_id', 'radar_channel').groupBy('ap_id', 'radar_channel').count()
df_radar_by_ap.show()


+------------+-------------+-----+
|       ap_id|radar_channel|count|
+------------+-------------+-----+
|5c5b3550032f|          116|    1|
|5c5b3550032f|           52|    3|
|5c5b3550032f|          132|    3|
|5c5b3550032f|          100|    4|
+------------+-------------+-----+



In [15]:
df_radar_by_channel = df_radar_in_site.select('radar_channel', 'last_radar_time').groupBy('radar_channel').agg({'last_radar_time':'max'})
df_radar_by_channel.show()

+-------------+--------------------+
|radar_channel|max(last_radar_time)|
+-------------+--------------------+
|          132|          1589037054|
|           52|          1589209332|
|          100|          1589019556|
|          116|          1588963397|
+-------------+--------------------+



In [16]:
# from pyspark.sql.types import *
# time_to_hour = udf(lambda t: int(t)//3600*3600, IntegerType())

df_radar_ch_by_hour= df_radar_in_site.select('hour').groupBy('hour').count()
df_radar_ch_by_hour.show()

+----+-----+
|hour|count|
+----+-----+
|  13|    1|
|   3|    2|
|   4|    1|
|   8|    3|
|   7|    1|
|  11|    2|
|   2|    1|
+----+-----+



In [17]:
# df_radar_ch_by_hour.groupBy("last_radar_time").count().show()

# from pyspark.sql.functions  import hour,minute,second
# # import org.apache.spark.sql.functions.{col,}
# # from pyspark.sql.functions import date_format
# # df_radar_in_site.withColumn("hour", hour(col("last_radar_time").cast(TimestampType))).show()
# df_radar_in_site.select(col("last_radar_time").cast(TimestampType))


# de-Weight function

In [18]:
"""


Radar_detected AP1

Wt_t1(ap1) = w_t0(ap2)* weight_decay(t1, site_last_seen) + weight(channel, map)
UNII-22 channel, weather_radio ,  



Neighbors: 
Wt_t1 (ap2)  = wt_t(ap2)  + weight_From_neighbor (rssi_ap1_ap2)  

Functions:

weight_decay(t1, last_seen) 
weight_0 * exp(- t/decay_lifetime),   decay_lifetime = 1 week,

weight(channel, map)
DFS Channel, 
Weather channel
 weight_From_neighbor (rssi) 
Weight_from_connectivity (SLE anomaly) 


Goal:  (org, site, ap, channel, weight_0, last_seen) in Redis

"""

import numpy as np
def weight_by_dfs_channel(chn, wt_0=0.4, wt_weather_ch=0.1 ):
    if 116<=chn<132:
        wt = wt_0 + wt_weather_ch
    else:
        wt = wt_0
    return wt

def weight_by_dist(dist_to_airport=100.0, max_wt=0.8):
    wt = 0.0
    if dist_to_airport< 10:
        wt = max_wt * np.exp(-dist_to_airport**2/5**2)
    
    return wt

def weight_impacted_neighbor(rssi, rssi_0 = -65):
    wt = 1.0
    if rssi < rssi_0:
        wt =  np.exp((rssi - rssi_0)/15) 
    return wt
    
    
def weight_tot(ch, rssi=-40, dist_to_airport=100.0, theta=[0.33, 0.33, 0.33]):
    wt=[0, 0, 0]
    wt[0] = weight_by_dfs_channel(ch)
    wt[1] = weight_by_dist(dist_to_airport)
    wt[2] = weight_impacted_neighbor(rssi)
    
    wt_0 = 0.0
    for i in range(2):
        wt_0 += wt[i] * theta[i]
    
#     print(wt_0, wt)
#     wt_0 = 1.0/(1.0 + np.exp(-0.6*wt_0))
    wt_0 = (wt[0] + wt[1]) * wt[2]

    return wt_0


In [19]:
ch = 40
d = 100
for r in range(-85, -50, 5):
    print(r, weight_tot(ch, r))

-85 0.10543885524629071
-80 0.14715177646857694
-75 0.2053668476130368
-70 0.2866125242295157
-65 0.4
-60 0.4
-55 0.4


In [21]:
for m in range(0, 20):
    print(m, "meter",  weight_by_dist(m))
    

0 meter 0.8
1 meter 0.7686315513218586
2 meter 0.6817150311729692
3 meter 0.5581410608568248
4 meter 0.42183393923443885
5 meter 0.2943035529371539
6 meter 0.18954220694569743
7 meter 0.112686736736836
8 meter 0.0618437923546398
9 meter 0.031331116079189654
10 meter 0.0
11 meter 0.0
12 meter 0.0
13 meter 0.0
14 meter 0.0
15 meter 0.0
16 meter 0.0
17 meter 0.0
18 meter 0.0
19 meter 0.0


In [22]:
for m in range(-40, -80, -5):
    print(m, "db",  weight_impacted_neighbor(m, -50))
    

-40 db 1.0
-45 db 1.0
-50 db 1.0
-55 db 0.7165313105737893
-60 db 0.513417119032592
-65 db 0.36787944117144233
-70 db 0.26359713811572677
-75 db 0.18887560283756183


In [23]:
# df_radar_in_site = df_radar.filter(col("site_id")==impacted_site_id)
# df_radar_in_site.groupBy("ap_id", col("pre_channel").alias("radar_channel")).agg({'radar_channel':'count', 'last_radar_time':'max'})
# df_radar_ap_in_site.show()

In [24]:
"""
Impacting on weight and  (Org, site, ap1, ap2, rssi ) () s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/
Radar_detected AP
Wt_t1(ap0) = w_t0(ap0)* weight_decay(t1, last_seen) + weight(channel, map)
Neighbors: 
Wt_t1 (ap1)  = wt_t(ap1)  + weight_From_neighbor (rssi)  
Functions:
weight_decay(t1, last_seen) 
weight_0 * exp(- t/decay_lifetime),   decay_lifetime = 1 week,
weight(channel, map)
DFS Channel, 
Weather channel
 weight_From_neighbor (rssi) 
Weight_from_connectivity (SLE anomaly) 
Goal:  (org, site, ap, channel, weight_0, last_seen) in Redis

"""

'\nImpacting on weight and  (Org, site, ap1, ap2, rssi ) () s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/\nRadar_detected AP\nWt_t1(ap0) = w_t0(ap0)* weight_decay(t1, last_seen) + weight(channel, map)\nNeighbors: \nWt_t1 (ap1)  = wt_t(ap1)  + weight_From_neighbor (rssi)  \nFunctions:\nweight_decay(t1, last_seen) \nweight_0 * exp(- t/decay_lifetime),   decay_lifetime = 1 week,\nweight(channel, map)\nDFS Channel, \nWeather channel\n weight_From_neighbor (rssi) \nWeight_from_connectivity (SLE anomaly) \nGoal:  (org, site, ap, channel, weight_0, last_seen) in Redis\n\n'

In [25]:
df_radar_in_site.show()

+------------+-------------+---------------+----+---------+
|       ap_id|radar_channel|last_radar_time|hour|dayofweek|
+------------+-------------+---------------+----+---------+
|5c5b3550032f|          100|     1588928836|   2|        4|
|5c5b3550032f|          100|     1588938523|   4|        4|
|5c5b3550032f|          100|     1588934289|   3|        4|
|5c5b3550032f|          132|     1588948689|   7|        4|
|5c5b3550032f|          132|     1589037054|   8|        5|
|5c5b3550032f|           52|     1589209332|   8|        0|
|5c5b3550032f|           52|     1588952779|   8|        4|
|5c5b3550032f|          132|     1588969154|  13|        4|
|5c5b3550032f|           52|     1588960942|  11|        4|
|5c5b3550032f|          116|     1588963397|  11|        4|
|5c5b3550032f|          100|     1589019556|   3|        5|
+------------+-------------+---------------+----+---------+



In [26]:
def weight_by_dfs_channel(chn, wt_0=0.4, wt_weather_ch=0.1 ):
    if 116<=chn<132:
        wt = wt_0 + wt_weather_ch
    else:
        wt = wt_0
    return wt

weight_channel = udf(lambda ch: weight_by_dfs_channel(ch, 0.6, 0.2))
df_radar_in_site_weight = df_radar_in_site.withColumn("weight", weight_channel(col('radar_channel')))
df_radar_in_site_weight= df_radar_in_site_weight.select("ap_id", "radar_channel", "weight").groupBy("ap_id", "radar_channel").agg({"weight":'max'})
df_radar_in_site_weight.show()


+------------+-------------+-----------+
|       ap_id|radar_channel|max(weight)|
+------------+-------------+-----------+
|5c5b3550032f|          116|        0.8|
|5c5b3550032f|           52|        0.6|
|5c5b3550032f|          132|        0.6|
|5c5b3550032f|          100|        0.6|
+------------+-------------+-----------+



# Impacts on neighbors

In [27]:
# rdd_ap_neigbhors = spark.sparkContext.textFile(ap_neighbors_path)
# rdd_ap_neigbhors.take(3)

['site,ap,ap2,band,time_epoch,rssi',
 '1916d52a-4a90-11e5-8b45-1258369c38a9,5c5b350e6064,5c5b350e06e0,24,1588294658,-61.0',
 '63b31b9a-e5c4-4613-8bd9-47ae12809b27,5c5b350ebe93,5c5b353e4e75,5,1588291686,-46.0']

In [28]:
df_scan = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(ap_neighbors_path)
df_scan.createOrReplaceTempView("scan_data")
df_Schema = df_scan.schema

Py4JJavaError: An error occurred while calling o270.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 100.0 failed 1 times, most recent failure: Lost task 2.0 in stage 100.0 (TID 4621, localhost, executor driver): org.apache.hadoop.fs.s3a.AWSS3IOException: getFileStatus on s3://mist-aggregated-stats-staging/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-05-01/hr=05/part-00000-057c5a1d-c34e-4844-a33a-69b175b1a3a8-c000.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: E4C6C3E478865076), S3 Extended Request ID: ZPDGKgvaEqND+Y5a/bHInqpr5pl3q+lBRt+buLzEpugGFSEniYLrL9EHu0M/U6UzkAK4p8tH8jE=: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: E4C6C3E478865076)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:178)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.ja

In [ ]:
df_scan.show()

In [ ]:
df_scan_in_site = df_scan.filter(col("site")==impacted_site_id)
df_scan_in_site.show()

In [ ]:

impact_aps = df_radar_in_site.select("ap_id").distinct().collect()

In [ ]:
impacted_ap_id = impact_aps[0]['ap_id']

In [ ]:
df_scan_neighbor = df_scan_in_site.filter((col("ap")==impacted_ap_id)&(col('rssi')>-75)&(col("ap2")!=''))\
                                .select('ap2', 'rssi').groupBy("ap2").agg({"rssi": 'max'})

df_scan_neighbor.show()

In [ ]:
# df_scan_neighbor.select('ap2', 'rssi').groupBy("ap2").agg({"rssi": 'max'}).show()
df_scan_neighbor = df_scan_neighbor.select(col("ap2").alias("ap"), col("max(rssi)").alias("max_rssi"))
df_scan_neighbor.show()

In [ ]:

weight_neighbor = udf(lambda r: weight_impacted_neighbor(r, -65))
df_scan_neighbor_weight = df_scan_neighbor.withColumn("weight", weight_channel(col('max_rssi')))

df_scan_neighbor_weight.show()
